In [1]:
%matplotlib notebook

import itertools
import random

import matplotlib.pyplot as plt
import pandas as pd
import scipy as sp
import seaborn as sns
import tqdm

# Summary of Riddler Express

It's your 30th birthday (congrats, by the way), and your friends bought you a cake with 30 candles on it. You make a wish and try to blow them out. Every time you blow, you blow out a random number of candles between one and the number that remain, including one and that other number. How many times do you blow before all the candles are extinguished, on average?

## computational solution

In [2]:
class Cake(object):
    def __init__(self, numCandles=30):
        self.numCandles = numCandles
        self.numLit = self.numCandles
        self.blows = 0

    def blow(self):
        self.numLit -= random.randint(1, self.numLit)
        self.blows += 1
        
    @property
    def blownOut(self):
        return self.numLit == 0

In [3]:
random.seed(1337)

def simul(numCandles=30):
    c = Cake(numCandles)
    while not c.blownOut:
        c.blow()
    return c.blows

In [4]:
numSimuls = 1e7
numCandles = 30

df = pd.DataFrame([{'n_blows': simul(numCandles)} for i in tqdm.trange(int(numSimuls))])
df.n_blows.mean()

100%|██████████| 10000000/10000000 [01:50<00:00, 90114.62it/s]


3.9948733999999999

## analytic solution

$$
\dfrac{\sum_{i=1}^N \langle 0 | n A^n |31\rangle}{N}
$$

Suppose $N=4$.

| after 1 | after 2 | after 3 | after 4 |
|---------|---------|---------|---------|
| 0       |         |         |         |
| 1       | 0       |         |         |
| 2       | 0       |         |         |
| 2       | 1       | 0       |         |
| 3       | 0       |         |         |
| 3       | 1       | 0       |         |
| 3       | 2       | 0       |         |
| 3       | 2       | 1       | 0       |

at first I thought it was just a $n$ choose $k$ issue, but the probability that we would end in states 3, 2, and 1 are each equal in the above, so they are not actually equally weighted in our sum.

Let 
$$
p(i_0, i_1) = \dfrac{1}{i_0}
$$
be the probability of going from $i_0$ candles to $i_1$ candles ($i_0 > i_1 >= 0$). 

To get to $0$ from $N$ in

+ 1 step: $p(N, 0) = \dfrac{1}{N}$
+ 2 steps: $\sum_{i=1}^N p(i, 0) = \sum_{i=1}^N \dfrac{1}{i}$

In [5]:
sum(1 / i for i in range(1, 31))

3.9949871309203906

# Summary of Riddler Classic

You and I stumble across a 100-sided die in our local game shop. We know we need to have this die — there is no question about it — but we're not quite sure what to do with it. So we devise a simple game: We keep rolling our new purchase until one roll shows a number smaller than the one before. Suppose I give you a dollar every time you roll. How much money do you expect to win?

*Extra credit*: What happens to the amount of money as the number of sides increases?

In [6]:
class DiceGame(object):
    def __init__(self, numSides=100):
        self.numSides = numSides
        self.numRolls = 0
        self.vals = [0]
        
    def roll(self):
        self.numRolls += 1
        r = random.randint(1, 100)
        self.vals.append(r)
        
    @property
    def game_over(self):
        return len(self.vals) > 1 and self.vals[-2] > self.vals[-1]
    
    
def one_dice_game(numSides=100):
    d = DiceGame(numSides)
    while not d.game_over:
        d.roll()
    return d.numRolls

In [7]:
numSimuls = 1e7
numSides = 100

df = pd.DataFrame([{'n_rolls': one_dice_game(numSides)} for i in tqdm.trange(int(numSimuls))])
df.n_rolls.mean()

100%|██████████| 10000000/10000000 [01:30<00:00, 110937.73it/s]


2.7321184000000001

In [10]:
sp.exp(1)

2.7182818284590451